# Modelos del lenguaje con n-gramas

En este cuaderno vamos a ver un ejemplo de cómo hacer un modelo del lenguaje con n-gramas usando la librería nltk y un corpus sobre recetas de cocina extraído de internet.

In [ ]:
!pip install nltk

Descargamos el corpus de recetas de cocina.

In [ ]:
!wget --no-check-certificate https://valencia.inf.um.es/dlpln/corpusRecetasv2.tgz
!tar -xzf corpusRecetasv2.tgz

Utilizando del Tokenizer de nltk creamos un modelo de bigramas y una función para generar texto a partir del modelo.

In [ ]:
import os
import nltk
from nltk.lm import Laplace
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.tokenize import word_tokenize, sent_tokenize

# Descargar los datos necesarios de NLTK si es la primera vez que se usa
nltk.download('punkt')

# Ruta de la carpeta que contiene los archivos de texto
ruta_corpus = "corpusRecetasv2"

# Leer cada archivo de texto en la carpeta, dividir en frases y tokenizar cada frase
corpus_frases = []
for archivo in os.listdir(ruta_corpus):
    if archivo.endswith(".txt"):  # Asegurarse de leer solo archivos de texto
        with open(os.path.join(ruta_corpus, archivo), "r", encoding="utf-8") as f:
            texto = f.read()
            # Dividir el texto en frases
            frases = sent_tokenize(texto.lower())
            # Tokenizar cada frase en palabras y agregar al corpus
            for frase in frases:
                tokens = word_tokenize(frase)
                corpus_frases.append(tokens)

# Definir el tamaño del n-grama (por ejemplo, trigramas)
n = 3

# Preparar los datos de entrenamiento con padding (inserta automáticamente los tokens <s> y </s>)
train_data, vocab = padded_everygram_pipeline(n, corpus_frases)

# Crear el modelo de lenguaje con suavizado de Laplace
modelo_trigramas = Laplace(n)
modelo_trigramas.fit(train_data, vocab)

# Función para generar texto a partir del modelo
def generar_texto(modelo, num_words, texto_inicial=None):
    contenido = texto_inicial or []
    for _ in range(num_words):
        siguiente_palabra = modelo.generate(text_seed=contenido[-(n-1):])  # usar (n-1) palabras anteriores
        contenido.append(siguiente_palabra)
        if siguiente_palabra == "</s>":  # detener si se genera un marcador de fin de oración
            break
    return ' '.join(contenido)




Una vez generado el modelo lo podemos guardar en disco

In [ ]:
import pickle

# Guardar el modelo en disco
with open('modelo_trigramas.pkl', 'wb') as archivo:
    pickle.dump(modelo_trigramas, archivo)
    print("Modelo de trigramas guardado.")

Podemos entonces cargar el modelo del disco

In [ ]:
# Cargar el modelo desde disco
with open('modelo_trigramas.pkl', 'rb') as archivo:
    modelo_trigramas_cargado = pickle.load(archivo)
    print("Modelo de trigramas cargado.")

En estas celdas hacemos pruebas de posibles frases que se pueden generar usando el modelo de trigramas.

In [ ]:
# Generar una frase de ejemplo
print("Frase generada:", generar_texto(modelo_trigramas, 40, texto_inicial=["pechuga", "de"]))

También podemos calcular la probabilidad de una palabra siguiente según un contexto dado.

In [ ]:
# Función para calcular la probabilidad de una palabra dada un contexto
def calcular_probabilidad(modelo, palabra, contexto):
    return modelo.score(palabra, contexto)

# Ejemplo: Calcular la probabilidad de "atún" y "queso" en el contexto "pasta con"
contexto = ["pasta", "con"]
palabra1 = "atún"
palabra2 = "queso"

prob_palabra1 = calcular_probabilidad(modelo_trigramas, palabra1, contexto)
prob_palabra2 = calcular_probabilidad(modelo_trigramas, palabra2, contexto)

print(f"Probabilidad de '{palabra1}' dado el contexto '{' '.join(contexto)}': {prob_palabra1}")
print(f"Probabilidad de '{palabra2}' dado el contexto '{' '.join(contexto)}': {prob_palabra2}")

Creamos un modelo de cuatrigramas

In [ ]:
# Definir el tamaño del n-grama (por ejemplo, trigramas)
n = 4

# Preparar los datos de entrenamiento con padding (inserta automáticamente los tokens <s> y </s>)
train_data, vocab = padded_everygram_pipeline(n, corpus_frases)

# Crear el modelo de lenguaje con suavizado de Laplace
modelo_cuatrigramas = Laplace(n)
modelo_cuatrigramas.fit(train_data, vocab)


Probamos con generar texto a partir de tres palabras con este nuevo modelo de cuatrigramas.

In [ ]:
# Generar una frase de ejemplo
print("Frase generada:", generar_texto(modelo_cuatrigramas, 40, texto_inicial=["calentar", "el", "agua"]))

Probamos ahora a generar la probabilidad de la siguiente palabra

In [ ]:
# Ejemplo: Calcular la probabilidad de "y" y "rojo" en el contexto "pasta con atún"
contexto = ["pasta", "con", "atún"]
palabra1 = "y"
palabra2 = "rojo"

prob_palabra1 = calcular_probabilidad(modelo_cuatrigramas, palabra1, contexto)
prob_palabra2 = calcular_probabilidad(modelo_cuatrigramas, palabra2, contexto)

print(f"Probabilidad de '{palabra1}' dado el contexto '{' '.join(contexto)}': {prob_palabra1}")
print(f"Probabilidad de '{palabra2}' dado el contexto '{' '.join(contexto)}': {prob_palabra2}")